In [1]:
import os, sys
import json
import pandas as pd
import numpy as np

from collections import defaultdict
from causallearn.utils.cit import CIT

/tmp/ipykernel_22529/337515062.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
sys.path.append('../submodules/pyCausalFS/pyCausalFS')
sys.path.append('../submodules/pyCausalFS/pyCausalFS/CBD')

In [5]:
#from MBs.semi_HITON.semi_HITON_PC import semi_HITON_PC
#from MBs.semi_HITON.semi_HITON_MB import semi_HITON_MB
from MBs.common.subsets import subsets
from MBs.common.condition_independence_test import cond_indep_test

In [6]:
tie_small_data = np.load('../data/tie_small.npy')
with open('../data/tie_small_names.json', 'r') as jf:
    tie_small_names = json.load(jf)

In [51]:
def find_best_parent(cit, node, candidates, existing_parents, threshold):
    """
    Find the best parent for a node among the candidate parents
    based on the G-squared (chi-squared) test with a threshold.
    """
    best_parents = []
    for candidate in candidates:
        if candidate != node and candidate not in existing_parents:
            p_value = cit(node, candidate, existing_parents)
            if p_value < threshold:
                best_parents.append(candidate)
    return best_parents

def semi_hiton_pc(data, labels, target, threshold=0.05):
    """
    Semi-Interleaved HITON-PC algorithm for learning the Markov boundary
    of a target variable from discrete data using G-squared test for independence.
    """
    # Step 1: Initialization
    markov_boundary = set()
    visited = set()
    cit = CIT(data, "gsq")
    indices = [i for i in range(len(labels))]
    target_idx = labels.index(target)
    queue = [target_idx]

    # Step 2: Forward phase
    while queue:
        node = queue.pop(0)
        visited.add(node)
        candidates = set(indices) - {node} - visited
        best_parents = find_best_parent(cit, node, candidates, markov_boundary, threshold)
        markov_boundary.update(best_parents)
        queue.extend(best_parents)

    print(markov_boundary)
    # Step 3: Backward phase
    while markov_boundary:
        node = markov_boundary.pop()
        candidates = set(indices) - {node} - markov_boundary
        best_parents = find_best_parent(cit, node, candidates, markov_boundary, threshold)
        markov_boundary.update(best_parents)
        print(markov_boundary)

    # Step 4: Remap to labels
    return markov_boundary

In [81]:
def find_dependencies(data, target, alpha, cit):
    dependencies = []
    for x in range(data.shape[1]):
        if x != target:
            #pval, dep = cond_indep_test(data, target, x, [])
            #print(pval, dep)
            pval = cit(target, x, None)
            if pval <= alpha:
                dependencies.append((x, pval))
    return sorted(dependencies, key=lambda x: x[1], reverse=False)

def find_parents(data, target, alpha, dependencies, cit):
    parents = []
    for x, dep in dependencies:
        parents.append(x)
        conditions = [i for i in parents if i != x]
        if len(conditions) >= 3:
            length = 3
        else:
            length = len(conditions)
        for j in range(length + 1):
            for s in subsets(conditions, j):
                #pval, _ = cond_indep_test(data, x, target, s)
                pval = cit(x, target, s)
                if pval > alpha:
                    parents.remove(x)
                    break
    return parents

def semi_HITON_PC(data, target, alpha, cit, is_discrete=True):
    dependencies = find_dependencies(data, target, alpha, cit)
    parents = find_parents(data, target, alpha, dependencies, cit)
    return parents, [], len(dependencies) + len(parents)

def semi_HITON_MB(data, target, alpha, is_discrete=True):
    cit = CIT(data, "gsq")
    TPC, _, ci_number = semi_HITON_PC(data, target, alpha, cit, is_discrete)
    MB = TPC.copy()
    for x in TPC:
        x_parents, _, ci_number2 = semi_HITON_PC(data, x, alpha, cit, is_discrete)
        ci_number += ci_number2
        for y in x_parents:
            if y != target and y not in TPC:
                condition_set = set()
                for z in MB:
                    condition_set.add(z)
                condition_set.update([x])
                #pval, _ = cond_indep_test(data, target, y, list(condition_set), is_discrete)
                pval = cit(target, y, list(condition_set))
                if pval <= alpha:
                    MB.append(y)
                    break
    return list(set(MB)), ci_number


In [100]:
import itertools

def contains_G_star(G_star, G):
    for g_star in G_star:
        if len(g_star) == 0:
            continue
        if g_star.issubset(G):
            return True
    return False

def get_embedded_dataset(data, G):
    idx_map = list(range(data.shape[1]))
    for i in G:
        idx_map[i] = -1
        for j in range(i + 1, data.shape[1]):
            idx_map[j] -= 1
    map = {}
    for i, j in enumerate(idx_map):
        map[i] = j
    imap = {j: i for i, j in map.items()}
    # Delete columns in G
    embed_data = np.delete(data, list(G), 1)
    return embed_data, map, imap

def check_independence_M_new(data, target, alpha, cit, M_new, M, G):
    W = set.intersection(*[M, M_new])
    S1 = M - M_new
    S2 = M_new - M
    for Y in S1:
        if cit(target, Y, list(S2)) > alpha:
            return False
    for Y in S2:
        if cit(target, Y, list(S1)) > alpha:
            return False
    return True
            
def tie_star(data, target, alpha, is_discrete=True):
    # Find the Markov Boundary of the entire dataset
    V = set(range(data.shape[1]))
    M, _ = semi_HITON_MB(data, target, alpha, is_discrete)
    print('M', M)
    M_queue = [set(M)]
    G_queue = [[]]
    G_star = [set()]
    cit = CIT(data, "gsq")
    
    # Now iterate to find other Markov boundaries from embedded distributions
    for i in range(1, len(V) - 1):
        for _G in itertools.combinations(V - {target}, i):
            print('_G', _G)
            if contains_G_star(G_star, _G):
                continue
            embed_data, map, imap = get_embedded_dataset(data, _G)
            # Get M_new from embedded distribution
            _M_new, _ = semi_HITON_MB(embed_data, map[target], alpha, is_discrete)
            # Remap to M_new
            M_new = set([imap[i] for i in _M_new])
            print('M_new', M_new)
            if check_independence_M_new(data, target, alpha, cit, set(M_new), set(M), _G):
                M_queue.append(set(M_new))
                G_queue.append(set(_G))
            else:
                G_star.append(set(_G))
            print('M_queue', M_queue)
            print('G_queue', G_queue)
            print('G_star', G_star)
    return M_queue                        

In [101]:
semi_HITON_MB(tie_small_data, 0, 0.05)

([1, 3], 13)

In [102]:
tie_star(tie_small_data, 0, 0.05)

M [1, 3]
_G (1,)
M_new {2, 3}
M_queue [{1, 3}]
G_queue [[]]
G_star [set(), {1}]
_G (2,)
M_new {1, 3}
M_queue [{1, 3}, {1, 3}]
G_queue [[], {2}]
G_star [set(), {1}]
_G (3,)
M_new {1}
M_queue [{1, 3}, {1, 3}, {1}]
G_queue [[], {2}, {3}]
G_star [set(), {1}]
_G (1, 2)
_G (1, 3)
_G (2, 3)
M_new {1}
M_queue [{1, 3}, {1, 3}, {1}, {1}]
G_queue [[], {2}, {3}, {2, 3}]
G_star [set(), {1}]


[{1, 3}, {1, 3}, {1}, {1}]

In [45]:
list(df.columns.values).index('T')

0

In [54]:
gsq = CIT(tie_small_data, "gsq")

In [55]:
gsq(0, [1,2], [3])

TypeError: '<' not supported between instances of 'int' and 'list'